In [18]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os
from datetime import datetime

In [19]:
# Read the tazs and 2021 Block groups
blockgroups = gpd.read_file(r'C:\Users\egreenwell\OneDrive - PlanRVA\Documents\GitHub\BG2021.shp')
tazs = gpd.read_file(r'C:\Users\egreenwell\OneDrive - PlanRVA\Documents\GitHub\TAZ.shp')

# Ensure both GeoDataFrames use the same CRS
blockgroups = blockgroups.to_crs(tazs.crs)

# Generate centroid points for block groups
blockgroups['centroid'] = blockgroups.geometry.centroid

# Create a new GeoDataFrame for the centroids
bgpoints = gpd.GeoDataFrame(blockgroups.drop(columns='geometry'), geometry='centroid', crs=blockgroups.crs)

# Perform the spatial join
join_df = gpd.sjoin(bgpoints, tazs, how="right", predicate="within")

# Drop geometry columns if they exist (for some reason this had to happen for it to reconcile the difference)
if 'geometry' in join_df.columns:
    join_df = join_df.drop(columns='geometry')
if 'centroid' in join_df.columns:
    join_df = join_df.drop(columns='centroid')

# Save the DataFrame without geometry columns to an Excel file
join_df.to_csv(r'C:\Users\egreenwell\OneDrive - PlanRVA\Documents\GitHub\join_df.csv')

In [20]:
#setting up definitions given
def sort_share_matrix(matrix):
    '''
    Sorts a share matrix first so that columns with ones higher up are further to the left

    Parameters
    ----------
    matrix (pandas.DataFrame):
        The share matrix to be sorted

    Returns
    -------
    sorted_matrix (pandas.DataFrame):
        Sorted share matrix
    '''
    first_one = pd.Series(index = matrix.columns) #Define series to identify the index of the first one in each column
    for col in matrix.columns:
        try:
            first_one[col] = list(matrix[col]).index(1) #Identify the index of the first one in the column
        except ValueError:
            continue #If there aren't any ones in the column, leave the value in `first_one` null

    order = first_one.sort_values().index #Find order of columns so that columns with ones higher up are further to the left
    return matrix[order]

def classify_nesting(nesting):
    '''
    Classifies a nesting as one to one, one TAZ to many Block Groups, one Block Group to Many TAZs, or many to many

    Parameters
    ----------
    nesting (length-2 tuple):
        Length-2 tuple where the first element is the number of TAZs and the second is the number of block groups

    Returns
    -------
    classification (str):
        Classification name
    '''
    if nesting[0] == 1 and nesting[1] == 1:
        return 'One TAZ = One Block Group'
    else:
        if nesting[0] == 1:
            return 'One TAZ = Many Block Groups'
        elif nesting[1] == 1:
            return 'Many TAZs = One Block Group'
        else:
            return 'Many TAZs = Many Block Groups'

def classify_pop(population):
    '''
    Classifies a geography based on population into three categories: less than 600, between 600 and 3000 (bounds included), and more than 3000

    Parameters
    ----------
    population (numeric):
        Population value to classify

    Returns
    -------
    class (str):
        Population classification
    '''
    if population < 600:
        return '<600'
    elif population <= 3000:
        return '600-3000'
    else:
        return '>3000'

In [22]:
#ensure files are read correctly

# Define the base file path
base_path = r'C:\Users\egreenwell\OneDrive - PlanRVA\Documents\GitHub'

# Get the directory for output files
start_time = datetime.now()
output_path = os.path.join(base_path, 'TAZ_BG_Nestings_Output_' + start_time.strftime('%y%m%d%H%M%S'))

# Create the output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# Define the input file path and ensure it is correct
infile = os.path.join(base_path, 'join_df.csv')
taz2bg_file = os.path.join(output_path, 'taz2bg.csv')
bg2taz_file = os.path.join(output_path, 'bg2taz.csv')

# Ensure the input file exists
if not os.path.isfile(infile):
    raise FileNotFoundError(f"The input file {infile} does not exist. Please check the file path.")

print('Reading Input File')
data = pd.read_csv(infile)

Reading Input File


In [29]:
# Check for missing values and drop rows with missing TAZ, GEOID, or TOTPOP values
data = data.dropna(subset=['TAZ', 'GEOID', 'TOTPOP'])


# Generate lists of unique TAZs and block groups
tazs = data['TAZ'].value_counts().sort_index().index  # List of TAZs
bgs = data['GEOID'].value_counts().sort_index().index  # List of block groups

n_t = len(tazs)  # Number of TAZs
n_b = len(bgs)  # Number of block groups

print('Creating Population and Share Matrices')

Creating Population and Share Matrices


In [30]:
# The population matrix has the TAZs as the rows and the block groups as the columns.
# Each entry is the population in the TAZ and the block group
pop_matrix = pd.DataFrame(np.zeros((n_t, n_b)), index=tazs, columns=bgs)
for row in data.index:
    taz = data.loc[row, 'TAZ']
    bg = data.loc[row, 'GEOID']
    pop = data.loc[row, 'TOTPOP']
    pop_matrix.loc[taz, bg] += pop

# The "Share Matrix" has the TAZs as the rows and the block groups as the columns
# It is equal to 1 if the TAZ and the Block Group share population, and 0 if they do not, indicating if a TAZ and block group share population
share_matrix = pop_matrix.astype(bool).astype(int)
share_matrix_backup = share_matrix.copy()

# The row and column sums of the population matrix are the TAZ and block group populations, respectively
taz_pops = pop_matrix.sum(1).astype(int)
bg_pops = pop_matrix.sum(0).astype(int)

print('Removing Zero-Population TAZs and Block Groups')

Removing Zero-Population TAZs and Block Groups


In [31]:
#Count the number of block groups sharing population with each TAZ and TAZs sharing population with block groups
bgs_in_tazs = share_matrix.sum(1)
tazs_in_bgs = share_matrix.sum(0)

#Identify TAZs and block groups with and without population
zero_pop_tazs = bgs_in_tazs[bgs_in_tazs == 0].index
pop_tazs = bgs_in_tazs[bgs_in_tazs > 0].index
zero_pop_bgs = tazs_in_bgs[tazs_in_bgs == 0].index
pop_bgs = tazs_in_bgs[tazs_in_bgs > 0].index

#Remove zero-population TAZs and block groups from the share matrix (they will be added later)
share_matrix = share_matrix.loc[pop_tazs, pop_bgs]

print('Sorting Share Matrix')

Sorting Share Matrix


In [32]:
sort_iter = 10 #Number of sorting iterations. Multiple are needed for correct sorting
for i in range(2*sort_iter): #In each iteration sort by block groups, than by TAZs
    share_matrix = sort_share_matrix(share_matrix)
    share_matrix = share_matrix.T

print('Identifying TAZ/Block Group Nestings')

C:\Users\egreenwell\AppData\Local\Temp\ipykernel_59008\2421278155.py:16: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  first_one = pd.Series(index = matrix.columns) #Define series to identify the index of the first one in each column


Identifying TAZ/Block Group Nestings


In [33]:
bgs = list(share_matrix.columns)
tazs = list(share_matrix.index)

n_bgs = len(bgs)
n_tazs = len(tazs)

bg_sums = share_matrix.sum(0)
taz_sums = share_matrix.sum(1)

nesting_id = 1
taz_index = 0
bg_index = 0

taz2nesting = pd.Series(index = tazs)
bg2nesting = pd.Series(index = bgs)
nestings = pd.DataFrame(columns = ['n_tazs', 'n_bgs'])

while taz_index < n_tazs or bg_index < min_bgs:

    current_taz = tazs[taz_index]
    current_bg = bgs[bg_index]

    #Identify the minimum number of block groups and TAZs needed in the nesting as the row and column sums of the share matrix for the current TAZ and block group
    min_bgs = share_matrix.loc[current_taz].sum()
    min_tazs = share_matrix[current_bg].sum()

    complete_nesting = False

    while not complete_nesting:

        more_tazs_needed = False
        more_bgs_needed = False

        #Check if more block groups need to be added to the nesting
        for taz in tazs[taz_index:(taz_index + min_tazs)]: #Check all of the current TAZs in the nesting
            test_row = share_matrix.loc[taz].copy() #Copy the row of the share matrix for testing
            test_row[bgs[bg_index:(bg_index + min_bgs)]] = 0 #Set the test row values to zero for the block groups already in the nesting
            if test_row.sum() > 0: #Check if there are additional block groups to be added, and if there are, add them
                more_bgs_needed = True
                min_bgs += test_row.sum()
                break

        #Now perform the same check, only add TAZs as necessary
        for bg in bgs[bg_index:(bg_index + min_bgs)]:
            test_col = share_matrix[bg].copy()
            test_col[tazs[taz_index:(taz_index + min_tazs)]] = 0
            if test_col.sum() > 0:
                more_tazs_needed = True
                min_tazs += test_col.sum()
                break

        #Now check if no TAZs or block groups were added. If they were, go back to the start of the while loop. If not, record the nesting.
        if not more_tazs_needed and not more_bgs_needed:
            complete_nesting = True

            taz2nesting[tazs[taz_index:(taz_index + min_tazs)]] = nesting_id
            bg2nesting[bgs[bg_index:(bg_index + min_bgs)]] = nesting_id

            nestings.loc[nesting_id] = [min_tazs, min_bgs]

            nesting_id += 1

            taz_index += min_tazs
            bg_index += min_bgs

print('Classifying TAZ/Block Group Nestings')

Classifying TAZ/Block Group Nestings


C:\Users\egreenwell\AppData\Local\Temp\ipykernel_59008\4254457838.py:14: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  taz2nesting = pd.Series(index = tazs)
C:\Users\egreenwell\AppData\Local\Temp\ipykernel_59008\4254457838.py:15: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  bg2nesting = pd.Series(index = bgs)


In [34]:
nestings['nesting'] = list(zip(nestings['n_tazs'], nestings['n_bgs'])) #Create tuples with # of TAZs and # of block groups
nestings['type'] = nestings['nesting'].apply(classify_nesting)

max_tazs = nestings['n_tazs'].max()
max_bgs = nestings['n_bgs'].max()

taz2nesting = pd.DataFrame(taz2nesting)
taz2nesting['nesting_id'] = taz2nesting[0]
del taz2nesting[0]
taz2nesting.index.name = 'taz_id'

taz2nesting = taz2nesting.reset_index()
taz2nesting['pop'] = taz2nesting['taz_id'].map(taz_pops)
taz2nesting['nest_type'] = taz2nesting['nesting_id'].map(nestings['type'])
taz2nesting['pop_class'] = taz2nesting['pop'].apply(classify_pop)
taz2nesting = taz2nesting.set_index('taz_id')

for taz in zero_pop_tazs:
    taz2nesting.loc[taz] = [np.nan, 0, 'No Population', '0']

for i in range(max_bgs):
    taz2nesting['blkgrp_%s'%(i+1)] = np.nan

for taz in taz2nesting.index:
    taz_nesting = taz2nesting.loc[taz, 'nesting_id']
    bgs_in_taz = list(bg2nesting[bg2nesting == taz_nesting].index)
    bgs_in_taz += (max_bgs - len(bgs_in_taz))*[np.nan]
    taz2nesting.loc[taz, ['blkgrp_%s'%(i+1) for i in range(max_bgs)]] = bgs_in_taz

bg2nesting = pd.DataFrame(bg2nesting)
bg2nesting['nesting_id'] = bg2nesting[0]
del bg2nesting[0]
bg2nesting.index.name = 'blkgrp_id'

bg2nesting = bg2nesting.reset_index()
bg2nesting['pop'] = bg2nesting['blkgrp_id'].map(bg_pops)
bg2nesting['nest_type'] = bg2nesting['nesting_id'].map(nestings['type'])
bg2nesting['pop_class'] = bg2nesting['pop'].apply(classify_pop)
bg2nesting = bg2nesting.set_index('blkgrp_id')

for bg in zero_pop_bgs:
    bg2nesting.loc[bg] = [np.nan, 0, 'No Population', '0']

for i in range(max_tazs):
    bg2nesting['TAZ_%s'%(i+1)] = np.nan

for bg in bg2nesting.index:
    bg_nesting = bg2nesting.loc[bg, 'nesting_id']
    tazs_in_bg = list(taz2nesting[taz2nesting['nesting_id'] == bg_nesting].index)
    tazs_in_bg += (max_tazs - len(tazs_in_bg))*[np.nan]
    bg2nesting.loc[bg, ['TAZ_%s'%(i+1) for i in range(max_tazs)]] = tazs_in_bg

del taz2nesting['nesting_id']
del bg2nesting['nesting_id']

print('Writing Output Files')
taz2nesting.sort_index().to_csv(taz2bg_file)
bg2nesting.sort_index().to_csv(bg2taz_file)

print('Go')

Writing Output Files
Go


C:\Users\egreenwell\AppData\Local\Temp\ipykernel_59008\2859427977.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bg2nesting['TAZ_%s'%(i+1)] = np.nan
C:\Users\egreenwell\AppData\Local\Temp\ipykernel_59008\2859427977.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  bg2nesting['TAZ_%s'%(i+1)] = np.nan
C:\Users\egreenwell\AppData\Local\Temp\ipykernel_59008\2859427977.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Cons